In [ ]:
!pip install scrapy

In [ ]:
import scrapy
from scrapy.crawler import CrawlerProcess
from scrapy.crawler import CrawlerRunner
from scrapy.utils.log import configure_logging
from scrapy.utils.project import get_project_settings

import json

from twisted.internet import reactor


In [ ]:
import requests
from parsel import Selector

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

# Original URL
url = f'https://www.olx.com.br/imoveis/aluguel/estado-rj/rio-de-janeiro-e-regiao/zona-sul?o=1'

# Proxy URL
proxy_api_key = "f7c54208-9521-4e93-99bd-db4937d17503"
proxy_url = f"https://proxy.scrapeops.io/v1/?api_key={proxy_api_key}&url={url}"

r = requests.get(proxy_url)



DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): proxy.scrapeops.io:443
DEBUG:urllib3.connectionpool:https://proxy.scrapeops.io:443 "GET /v1/?api_key=f7c54208-9521-4e93-99bd-db4937d17503&url=https://www.olx.com.br/imoveis/aluguel/estado-rj/rio-de-janeiro-e-regiao/zona-sul?o=1 HTTP/1.1" 200 None


In [ ]:
s = Selector(text= r.text)

s.xpath('//script[@id="__NEXT_DATA__"]/text()')

[<Selector query='//script[@id="__NEXT_DATA__"]/text()' data='{"props":{"pageProps":{"category":"im...'>]

In [ ]:

itens =json.loads(s.xpath('//script[@id="__NEXT_DATA__"]/text()').get())


In [ ]:
itens


{'props': {'pageProps': {'category': 'imoveis',
   'filtersTemplate': {'slug': 'filters_1002_v4',
    'template': [{'id': 'realestatetype_1002',
      'type': 'checkbox',
      'owner': 'buyersjourney@olxbr.com',
      'props': {'name': 'category',
       'label': 'Tipo do imóvel',
       'behavior': 'multi_selection',
       'linkable': False,
       'datasource': {'type': 'local',
        'values': [{'name': '1020',
          'label': 'Apartamentos para alugar',
          'value': '1020',
          'extraData': {'friendlyPath': 'apartamentos'}},
         {'name': '1040',
          'label': 'Casas para alugar',
          'value': '1040',
          'extraData': {'friendlyPath': 'casas'}},
         {'name': '1060',
          'label': 'Quartos para alugar',
          'value': '1060',
          'extraData': {'friendlyPath': 'aluguel-de-quartos'}}]},
       'queryStrings': {'friendly_qs_order': '1',
        'strategy': '',
        'values': [{'qs': 'ret', 'order': '32'}]}}},
     {'id': 'r

In [ ]:
print(itens['props']['pageProps']['ads'][4]['properties'][2]['label'])
print(itens['props']['pageProps']['ads'][4]['properties'][2]['value'])

Condomínio
R$ 680


In [ ]:
a = itens.get('props').get('pageProps').get('ads')[4].get('properties')[2].get('value')
print(a)

R$ 680


In [ ]:
print(itens['props']['pageProps']['ads'][4]['properties'][3]['label'])
print(itens['props']['pageProps']['ads'][4]['properties'][3]['value'])

IPTU
R$ 96


In [ ]:
print(itens['props']['pageProps']['ads'][4]['properties'][4]['label'])
print(itens['props']['pageProps']['ads'][4]['properties'][4]['value'])

Área útil
28m²


In [ ]:
print(itens['props']['pageProps']['ads'][4]['properties'][5]['label'])
print(itens['props']['pageProps']['ads'][4]['properties'][5]['value'])

Quartos
1


In [ ]:
print(itens['props']['pageProps']['ads'][4]['properties'][6]['label'])
print(itens['props']['pageProps']['ads'][4]['properties'][6]['value'])

Banheiros
1


In [ ]:
print(itens['props']['pageProps']['ads'][2]['properties'][7]['label'])
print(itens['props']['pageProps']['ads'][2]['properties'][7]['value'])

Vagas na garagem
0


In [ ]:
# OLX - Scrape OLX Brasil website

# Define the spider
class OlxHouses(scrapy.Spider):
    name = 'olx'

    custom_settings = {
        'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36',
        'AUTOTHROTTLE_ENABLED': True,
        'LOG_LEVEL': 'ERROR',  # Suppress logs for cleaner output
        'DOWNLOAD_DELAY': 2,  # 2 seconds delay between requests
        'FEEDS': {'olx_houses.json':
                {
                'format': 'json',
                'encoding': 'utf8',
                'store_empty': False,
                'indent': 4,
                'overwrite': True,  # Overwrite existing file if any
                }
                  }
                      }


    def start_requests(self):
        for page in range(1, 90):
            # Original URL
            url = f'https://www.olx.com.br/imoveis/aluguel/estado-rj/rio-de-janeiro-e-regiao/zona-sul?o={page}'
            # Proxy URL
            proxy_api_key = "37dba935-88bc-4998-88af-af7a7460c885"
            proxy_url = f"https://proxy.scrapeops.io/v1/?api_key={proxy_api_key}&url={url}"
            yield scrapy.Request(proxy_url)

    def parse(self, response, **kwargs):
        html = json.loads(response.xpath('//script[@id="__NEXT_DATA__"]/text()').get())
        houses = html.get('props').get('pageProps').get('ads')
        for house in houses:
            yield {
                'titulo': house.get('title'),
                'preco_aluguel': house.get('price'),
                'localizacao': house.get('location'),
                'bairro': house.get('locationDetails', {}).get('neighbourhood'),
                'area': next((prop['value'] for prop in house.get('properties', []) if prop['name'] == 'size'), None),
                'iptu': next((prop['value'] for prop in house.get('properties', []) if prop['name'] == 'iptu'), None),
                'preco_condominio': next((prop['value'] for prop in house.get('properties', []) if prop['name'] == 'condominio'), None),
                'quartos': next((prop['value'] for prop in house.get('properties', []) if prop['name'] == 'rooms'), None),
                'banheiros': next((prop['value'] for prop in house.get('properties', []) if prop['name'] == 'bathrooms'), None),
                'garagem': next((prop['value'] for prop in house.get('properties', []) if prop['name'] == 'garage_spaces'), None),

                   }

# Run the spider using CrawlerProcess instead of CrawlerRunner and reactor

# Check if the process has already been started
if 'process' not in globals() or not isinstance(globals()['process'], scrapy.crawler.CrawlerProcess):
  process = CrawlerProcess(get_project_settings())
  process.crawl(OlxHouses)
  process.start() # the script will block here until the crawling is finished


A saída de streaming foi truncada nas últimas 5000 linhas.
DEBUG:scrapy.core.scraper:Scraped from <200 https://proxy.scrapeops.io/v1/?api_key=f7c54208-9521-4e93-99bd-db4937d17503&url=https://www.olx.com.br/imoveis/aluguel/estado-rj/rio-de-janeiro-e-regiao/zona-sul?o=30>
{'titulo': 'Duplex para venda e locação tem 233 metros quadrados com 3 quartos em Flamengo - Rio de Ja', 'preco_aluguel': 'R$ 12.000', 'localizacao': 'Rio de Janeiro, Flamengo', 'bairro': 'Flamengo', 'area': '233m²', 'iptu': 'R$ 797', 'preco_condominio': 'R$ 1.510', 'quartos': '3', 'banheiros': '4', 'garagem': '1'}
DEBUG:scrapy.core.scraper:Scraped from <200 https://proxy.scrapeops.io/v1/?api_key=f7c54208-9521-4e93-99bd-db4937d17503&url=https://www.olx.com.br/imoveis/aluguel/estado-rj/rio-de-janeiro-e-regiao/zona-sul?o=30>
{'titulo': 'JOA', 'preco_aluguel': 'R$ 3.000', 'localizacao': 'Rio de Janeiro, São Conrado', 'bairro': 'São Conrado', 'area': '110m²', 'iptu': None, 'preco_condominio': None, 'quartos': '2', 'banheiro

In [ ]:
# Saving as Json

with open("olx_houses.json", "r") as f:
    data = json.load(f)
    print(json.dumps(data, indent=4, ensure_ascii=False))

[
    {
        "titulo": "ALUGO 2 QUARTOS NO CATETE COM GARAGEM",
        "preco_aluguel": "R$ 2.800",
        "localizacao": "Rio de Janeiro, Catete",
        "bairro": "Catete",
        "area": "80m²",
        "iptu": "R$ 350",
        "preco_condominio": "R$ 1.740",
        "quartos": "2",
        "banheiros": "2",
        "garagem": "1"
    },
    {
        "titulo": "Apartamento 3 quartos, Copacabana.",
        "preco_aluguel": "R$ 6.000",
        "localizacao": "Rio de Janeiro, Copacabana",
        "bairro": "Copacabana",
        "area": "146m²",
        "iptu": "R$ 720",
        "preco_condominio": "R$ 1.610",
        "quartos": "3",
        "banheiros": "2",
        "garagem": "1"
    },
    {
        "titulo": "Aluguel mensal em Pousada Boutique - Santa Teresa",
        "preco_aluguel": "R$ 3.200",
        "localizacao": "Rio de Janeiro, Santa Teresa",
        "bairro": "Santa Teresa",
        "area": "20m²",
        "iptu": "R$ 0",
        "preco_condominio": "R$ 1",
       

In [ ]:

# Save the file to Google Drive
with open("/content/drive/My Drive/olx_houses.json", "w") as f:
    json.dump(data, f, indent=4, ensure_ascii=False)
